In [98]:
import pandas as pd
import itertools
from collections import Counter
import numpy as np
from matplotlib import pyplot as plt
import nltk
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
import re
import string
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer  
from nltk.corpus import words
from copy import deepcopy
from gensim.models import Word2Vec
from sklearn import svm

import tensorflow as tf
from tensorflow.keras import Sequential,optimizers
from tensorflow.keras.layers import Dense,BatchNormalization,Dropout,Activation
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
import tensorflow_hub as hub


In [3]:
def preproc_text(text):
    text = re.sub(r"\d+", "", text.lower())
    for i in string.punctuation:
        text = text.replace(i,"")
    stop_words = set(stopwords.words("english"))
    tokens = word_tokenize(text)
    sample = [i for i in tokens if not i in stop_words]
    # lancaster_
    stemmer = LancasterStemmer() 
    lemmatizer = WordNetLemmatizer()

    
    
    return [lemmatizer.lemmatize(word) for word in sample]

In [4]:
data = pd.read_csv("movie_data.csv")
data.head()
review = data["review"] # create a data frame with only data 
review = review.apply(preproc_text)

In [84]:
model = Word2Vec(review[:40000], vector_size=500, window=10, min_count=10)

In [85]:
dic = set(model.wv.index_to_key)
def generate_vec(wordlist):
    feature = np.zeros(500)
    count = 0
    for word in wordlist:
        if word in dic:
            feature = np.add(model.wv[word], feature)
            count += 1
    
    return list(feature/count)
            

In [86]:
def split_train(original_train_data, size=10000):
    return original_train_data[:size], original_train_data[40000:40000+2000]

def binary_transform(sentiment):
    if sentiment == "positive":
        return 1
    return 0

In [87]:
label = data["sentiment"].apply(binary_transform)

In [88]:
train_X, val_X = split_train(review)
train_y, val_y = split_train(label)

In [89]:
train_X = train_X.apply(generate_vec).values

In [90]:
val_X = val_X.apply(generate_vec).values

In [91]:
X = np.zeros(10000*500).reshape(10000,500)
Y = np.array(train_y)
for i in range(5000):
    X[i] = train_X[i]

In [93]:
x_vt = np.zeros(2000*500).reshape(2000,500)
y = np.array(val_y)
for i in range(2000):
    x_vt[i] = val_X[i]

In [94]:
val_X, test_X = x_vt[:1000], x_vt[1000:]
val_y, test_y = y[:1000], y[1000:]


In [101]:
model_2 = Sequential([
            Dense(16, input_dim=500, activation="relu"),
            #Dense(64, activation="relu"),
            Dense(32, activation="relu"),
            #Dense(64, activation="relu", kernel_regularizer=regularizers.l2(0.0001)),
            Dense(1, activation="sigmoid")
        ])

model_2.compile(
            optimizer="Adam",
            loss="binary_crossentropy",
            metrics=["binary_accuracy"]
        )

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=3, min_lr=0.001)

es = EarlyStopping(monitor='val_loss',
                   min_delta=0, patience=2, verbose=1, mode='auto',
                   baseline=None, restore_best_weights=False)


In [102]:
history = model_2.fit(
            x=X,
            y=Y,
            batch_size=32,
            epochs=300,
            verbose=1,
            validation_data=(val_X, val_y),
            callbacks=[reduce_lr, es],
            shuffle=True
        )


Epoch 1/300
313/313 [==============================] - 1s 2ms/step - loss: 0.6151 - binary_accuracy: 0.5998 - val_loss: 0.3450 - val_binary_accuracy: 0.8550
Epoch 2/300
313/313 [==============================] - 0s 810us/step - loss: 0.5086 - binary_accuracy: 0.6828 - val_loss: 0.3364 - val_binary_accuracy: 0.8610
Epoch 3/300
313/313 [==============================] - 0s 786us/step - loss: 0.5102 - binary_accuracy: 0.6796 - val_loss: 0.3454 - val_binary_accuracy: 0.8670
Epoch 4/300
313/313 [==============================] - 0s 786us/step - loss: 0.5045 - binary_accuracy: 0.6882 - val_loss: 0.3391 - val_binary_accuracy: 0.8620
Epoch 00004: early stopping


In [104]:
prediction = model_2.predict(test_X)
        
for i in range(len(prediction)):
    if prediction[i] <= 0.5:
        prediction[i] = 0
    else:
        prediction[i] = 1
        
error_count = 0 
for i in range(len(prediction)):
    if prediction[i] != test_y[i]:
        error_count += 1

print("Error Count: {}".format(error_count))
print("Accuracy: {}%".format(((len(test_y)-error_count)/len(test_y))*100))

Error Count: 154
Accuracy: 84.6%
